### Company X Biased Predictor Analysis

#### Imports

In [151]:
#Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#AIF360 Dataset Classes 
from aif360.datasets import StandardDataset
from aif360.datasets import BinaryLabelDataset

#AIF360 Metrics Classes  
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric

#AIF360 Algorithms 
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, EqOddsPostprocessing
# AI360 Explainers
from aif360.explainers import MetricTextExplainer

#Sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,train_test_split

#Tensorflow -- install version before 2.0
import tensorflow as tf 

#For Markdown
from IPython.display import Markdown, display

#Helper Functions
from src.classifier_functions import *

#### Load Data


In [55]:
data = pd.read_csv('../company_x.csv', index_col='employee_id')

In [56]:
data.head()


,signing_bonus,salary,degree_level,sex,yrs_experience,dept,is_manager,direct_reports,boss_id,total_reports,company_level
employee_id,,,,,,,,,,,
138719,0,273000.0,Master,M,2,engineering,False,0.0,43602,0,0
3192,0,301000.0,Bachelor,F,1,sales,False,0.0,87847,0,0
114657,0,261000.0,Master,F,2,sales,False,0.0,180854,0,0
29039,0,86000.0,High_School,F,4,HR,False,0.0,88370,0,0
118607,0,126000.0,Bachelor,F,3,sales,False,0.0,23565,0,0


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 138719 to 72227
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   signing_bonus   10000 non-null  int64  
 1   salary          10000 non-null  float64
 2   degree_level    10000 non-null  object 
 3   sex             10000 non-null  object 
 4   yrs_experience  10000 non-null  int64  
 5   dept            10000 non-null  object 
 6   is_manager      10000 non-null  bool   
 7   direct_reports  10000 non-null  float64
 8   boss_id         10000 non-null  int64  
 9   total_reports   10000 non-null  int64  
 10  company_level   10000 non-null  int64  
dtypes: bool(1), float64(2), int64(5), object(3)
memory usage: 869.1+ KB


In [113]:
## Create Label 
data_with_label = data.copy()
data_with_label['salary'] = data_with_label['salary'].transform(lambda x: x > 100000).astype(int)
data_with_label['sex'] = data_with_label['sex'].transform(lambda x: x == 'M').astype(int)
data_with_label.head()

,signing_bonus,salary,degree_level,sex,yrs_experience,dept,is_manager,direct_reports,boss_id,total_reports,company_level
employee_id,,,,,,,,,,,
138719,0,1,Master,1,2,engineering,False,0.0,43602,0,0
3192,0,1,Bachelor,0,1,sales,False,0.0,87847,0,0
114657,0,1,Master,0,2,sales,False,0.0,180854,0,0
29039,0,0,High_School,0,4,HR,False,0.0,88370,0,0
118607,0,1,Bachelor,0,3,sales,False,0.0,23565,0,0


In [114]:
### Create StandardDataset

std_data = StandardDataset(df=data_with_label,   
                         label_name='salary',
                         favorable_classes =[1],
                        protected_attribute_names=['sex'], 
                         privileged_classes=[[1]],
                        categorical_features=['degree_level', 'dept'], 
                          features_to_drop=['boss_id', 'is_manager'])



In [115]:
df_data = std_data.convert_to_dataframe()
binary_data = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=df_data[0], label_names=['salary'],
              protected_attribute_names=['sex'])

privileged_groups= [{'sex':1}]
unprivileged_groups= [{'sex': 0}]

In [116]:
## Splitting the Data
data_train, data_vt = binary_data.split([0.7], shuffle=True)

data_val, data_test = data_vt.split([0.5], shuffle=True)

In [117]:
### First Metrics
metric_train = BinaryLabelDatasetMetric(data_train, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)
#Explainer 
ex_metric_train = MetricTextExplainer(metric_train)

print(ex_metric_train.mean_difference())
print('\n')
print(ex_metric_train.disparate_impact())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.1461893846888619


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.8258243465060383


### Random Forest Classifier 

In [165]:
#Creating X,y
X_train = data_train.features
y_train = data_train.labels.ravel()
X_test = data_val.features
y_test = data_val.labels.ravel()

In [119]:
# Creating Random Forest Classifier
rfc = RandomForestClassifier()
# #rf_random = RandomizedSearchCV(estimator = rfc, 
#                                param_distributions = random_grid, 
#                                n_iter = 100, cv = 5, verbose=5, 
#                                random_state=42, n_jobs = -1)

In [120]:
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(3,10,7)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree

# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf}
# print(random_grid)

In [121]:
# Fit the random search modelrfc.fit(X_train, y_train)
#rf_random.fit(X_train, y_train)
# rf_random.best_params_

In [166]:
rfc = RandomForestClassifier(n_estimators=500, 
                             min_samples_split=10,
                             min_samples_leaf=1, 
                             max_features='sqrt',
                            max_depth =5)

In [167]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [168]:
#Creating Predicted Dataframe
y_pred = rfc.predict(X_test)
data_val_pred = data_val.copy()
data_val_pred.labels = y_pred.ravel()

In [156]:
#Prep dataset to test out Aequitas 
# df = data_val.convert_to_dataframe()[0]
# df.head()
# aeq_df = pd.DataFrame()
# aeq_df['score'] = y_pred
# aeq_df['label_value'] = df['salary'].values
# aeq_df['sex'] = df['sex'].values
# aeq_df['sex'] = aeq_df['sex'].transform(lambda x: 'Male' if x == 1 else 'Female') 
# aeq_df.to_csv('../aeq.csv', index=False)
# aeq_df.head()

In [157]:
rfc.score(X_test, y_test)

0.8826666666666667

In [172]:
data_val_pred

array([1., 0., 0., ..., 1., 0., 1.])

In [158]:
rfc.predict_proba(X_test)[:10]

array([[0.13043594, 0.86956406],
       [0.74166198, 0.25833802],
       [0.72040179, 0.27959821],
       [0.11563695, 0.88436305],
       [0.11563178, 0.88436822],
       [0.10897434, 0.89102566],
       [0.12653249, 0.87346751],
       [0.11918074, 0.88081926],
       [0.08710609, 0.91289391],
       [0.11515705, 0.88484295]])

In [159]:
#Getting the Metrics 
metric_preds = BinaryLabelDatasetMetric(data_val_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_metric_preds = MetricTextExplainer(metric_preds)

print(ex_metric_preds.mean_difference())
print('\n')
print(ex_metric_preds.disparate_impact())


model_metric = ClassificationMetric(data_val, data_val_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_model_metric= MetricTextExplainer(model_metric)

print('\n')

print(ex_model_metric.average_odds_difference())
print('\n')
print(ex_model_metric.equal_opportunity_difference())
print('\n')
print(ex_model_metric.theil_index())


print(ex_model_metric.precision(privileged=True))
print(ex_model_metric.precision(privileged=False))
print(ex_model_metric.recall(privileged=True))
print(ex_model_metric.recall(privileged=False))


Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.1731013749938397


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.8066838385839772


Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.16692926345609418


True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.04584209866395472


Theil index (generalized entropy index with alpha = 1): 0.056627397722064106
Positive predictive value on privileged instances: 0.8900226757369615
Positive predictive value on unprivileged instances: 0.8978494623655914
True positive rate on privileged instances: 0.9788029925187033
True positive rate on unprivileged instances: 0.9329608938547486


In [111]:
scorecard(y_test, y_pred)

The Accuracy score is 0.830.

The Precision score is 0.803.

The Recall score is 0.991.

      Confusion Matrix


,Actual True,Actual False
Predicted True,1001,246
Predicted False,9,244


In [74]:
data_val.labels.shape

(1500, 1)

In [75]:
data_val_pred.labels.shape

(1500,)

## Debiasing through Preprocessing 

#### Reweighing

In [76]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, 
               privileged_groups=privileged_groups)

#Splitting the Data
data_rw_train = RW.fit_transform(data_train)

data_rw_val = RW.fit_transform(data_val)

In [130]:
### Reweigh Metrics
metric_rw_train = BinaryLabelDatasetMetric(data_rw_train, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)
#Explainer 
ex_metric_rw_train = MetricTextExplainer(metric_rw_train)

print(ex_metric_rw_train.mean_difference())
print('\n')
print(ex_metric_rw_train.disparate_impact())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -4.440892098500626e-16


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.9999999999999994


### Random Forest Classifier with Reweighing 

In [131]:
#Creating X,y
X_rw_train = data_rw_train.features
y_rw_train = data_rw_train.labels.ravel()
X_rw_test = data_rw_val.features
y_rw_test = data_rw_val.labels.ravel()

In [132]:
# Creating Random Forest Classifier
rfc_rw = RandomForestClassifier(n_estimators=1000, max_depth=5)
rfc_rw.fit(X_rw_train, y_rw_train, sample_weight=data_rw_train.instance_weights)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [133]:
#Creating Predicted Dataframe
y_rw_pred = rfc_rw.predict(X_test)
data_val_rw_pred = data_rw_val.copy()
data_val_rw_pred.labels = y_rw_pred

In [134]:
#Getting the Metrics 
display(Markdown("#### Model - with reweighing - dataset metrics"))


metric_rw_preds = BinaryLabelDatasetMetric(data_val_rw_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_metric_rw_preds = MetricTextExplainer(metric_rw_preds)

print(ex_metric_rw_preds.mean_difference())
print('\n')
print(ex_metric_rw_preds.disparate_impact())

display(Markdown("#### Model - with reweighing - classification metrics"))
model_rw_metric = ClassificationMetric(data_rw_val, data_val_rw_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_model_rw_metric= MetricTextExplainer(model_rw_metric)

print(ex_model_rw_metric.average_odds_difference())
print(ex_model_rw_metric.equal_opportunity_difference())
print(ex_model_rw_metric.theil_index())
print(ex_model_rw_metric.precision(privileged=True))
print(ex_model_rw_metric.precision(privileged=False))
print(ex_model_rw_metric.recall(privileged=True))
print(ex_model_rw_metric.recall(privileged=False))

#### Model - with reweighing - dataset metrics

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): 0.005402156724691465


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 1.006487525561851


#### Model - with reweighing - classification metrics

True positive rate (TPR, recall, sensitivity = TP / (TP + FN)): 0.8292135549872123
Positive predictive value (PPV, precision = TP / (TP + FP)): 0.7842587177108318
Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): 0.019407656928970007
True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.004795396419437159
Theil index (generalized entropy index with alpha = 1): 0.19456277933687138
Positive predictive value on privileged instances: 0.7876111957393863
Positive predictive value on unprivileged instances: 0.7780181241960171
True positive rate on privileged instances: 0.8308823529411764
True positive rate on unprivileged instances: 0.8260869565217392


In [82]:
scorecard(y_rw_test, y_rw_pred)

The Accuracy score is 0.898.

The Precision score is 0.911.

The Recall score is 0.965.

      Confusion Matrix


,Actual True,Actual False
Predicted True,1143,112
Predicted False,41,204


### Inprocessing with Adversarial Debiaising



In [135]:
#sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [136]:
##Creating Tensorflow Session - must not use Tensorflow 2.0 
sess = tf.Session()
db_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                            num_epochs  = 100,
                            batch_size = 50,
                            adversary_loss_weight = .1,     
                            debias=True,
                          sess=sess)

In [137]:
db_model.fit(data_train) 

epoch 0; iter: 0; batch classifier loss: 0.744275; batch adversarial loss: 0.716503
epoch 1; iter: 0; batch classifier loss: 0.405986; batch adversarial loss: 0.692637
epoch 2; iter: 0; batch classifier loss: 0.282487; batch adversarial loss: 0.679347
epoch 3; iter: 0; batch classifier loss: 0.276106; batch adversarial loss: 0.622846
epoch 4; iter: 0; batch classifier loss: 0.273404; batch adversarial loss: 0.575941
epoch 5; iter: 0; batch classifier loss: 0.320075; batch adversarial loss: 0.668837
epoch 6; iter: 0; batch classifier loss: 0.293012; batch adversarial loss: 0.692117
epoch 7; iter: 0; batch classifier loss: 0.570814; batch adversarial loss: 0.609986
epoch 8; iter: 0; batch classifier loss: 0.281676; batch adversarial loss: 0.671186
epoch 9; iter: 0; batch classifier loss: 0.252646; batch adversarial loss: 0.552042
epoch 10; iter: 0; batch classifier loss: 0.311970; batch adversarial loss: 0.567946
epoch 11; iter: 0; batch classifier loss: 0.172832; batch adversarial loss:

In [138]:
dataset_debiasing_train = db_model.predict(data_train)
dataset_debiasing_test = db_model.predict(data_val)

In [139]:

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(data_val, 
                                                 dataset_debiasing_test,
                                                    unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())
                                                        
                                                        

#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.199540
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.163393


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.879333
Test set: Balanced classification accuracy = 0.776445
Test set: Disparate impact = 0.817526
Test set: Equal opportunity difference = -0.045842
Test set: Average odds difference = -0.151006
Test set: Theil_index = 0.057463


In [140]:
scorecard(data_val.labels, dataset_debiasing_test.labels)

The Accuracy score is 0.879.

The Precision score is 0.889.

The Recall score is 0.965.

      Confusion Matrix


,Actual True,Actual False
Predicted True,1119,140
Predicted False,41,200


### Postprocessing with Equalized Odds Postprocessing

In [141]:

# Learn parameters to equalize odds and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint='fpr')
cpp = cpp.fit(data_val, data_val_pred)

In [162]:
cpp_pred = cpp.predict(data_val)


cpp_metric = ClassificationMetric(data_val, cpp_pred,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

In [163]:
print(cpp_metric.generalized_false_positive_rate())

print(cpp_metric.generalized_false_negative_rate())

0.0
0.0


In [164]:
scorecard(data_val.labels, cpp_pred.labels)

The Accuracy score is 1.000.

The Precision score is 1.000.

The Recall score is 1.000.

      Confusion Matrix


,Actual True,Actual False
Predicted True,1160,0
Predicted False,0,340


In [161]:
# Learn parameters to equalize odds and apply to create a new dataset
eop = EqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     )
eop = eop.fit(data_val, data_val_pred)

eop_pred = eop.predict(data_val)


eop_metric = ClassificationMetric(data_val, eop,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

TypeError: 'classified_dataset' should be a BinaryLabelDataset.

In [149]:
print(cpp_metric.generalized_false_positive_rate())

print(cpp_metric.generalized_false_negative_rate())

0.0
0.0
